### 注意
解説のわかりやすさのため一部で hive_metastore を利用する & ストレージを直接参照する ため本ノートブックの実行は汎用クラスタを利用してください（いずれのオペレーションもセキュリティ観点で非推奨であり Serverless からのオペレーションはサポートされないため）。


★ わかりやすくするために一時的に hive を利用  
★ 自動optimizzeを無効化  
https://learn.microsoft.com/ja-jp/azure/databricks/delta/tune-file-size#auto-optimize  
https://learn.microsoft.com/ja-jp/azure/databricks/sql/language-manual/sql-ref-syntax-ddl-tblproperties

In [0]:
%run .././include/handson.h

In [0]:
%python
# SQL コンテキスト用パラメータ定義（spark.conf.set はサーバレス未サポートのため Widgets でパラメータ定義する）
dbutils.widgets.text("sample_dataset_path", sample_dataset_path)
dbutils.widgets.text("your_catalog", your_catalog)
dbutils.widgets.text("your_schema", your_schema)

# Delta Lake の構造

In [0]:
USE CATALOG hive_metastore;
CREATE SCHEMA IF NOT EXISTS ${your_schema};
USE SCHEMA ${your_schema};

## ヒストリーを持つDeltaテーブルの作成

このクエリの実行を待っている間、実行されているトランザクションの合計数を特定できるか試してみてください。

In [0]:
DROP TABLE IF EXISTS students;
CREATE OR REPLACE TABLE students
  (id INT, name STRING, value DOUBLE)
--  TBLPROPERTIES(delta.autoOptimize.autoCompact = false, delta.autoOptimize.optimizeWrite = false) -- 10.4 LTS 以降は無効不可
;

INSERT INTO students VALUES (1, "Yve", 1.0);
INSERT INTO students VALUES (2, "Omar", 2.5);
INSERT INTO students VALUES (3, "Elia", 3.3);

INSERT INTO students
VALUES 
  (4, "Ted", 4.7),
  (5, "Tiffany", 5.5),
  (6, "Vini", 6.3);
  
UPDATE students 
SET value = value + 1
WHERE name LIKE "T%";

DELETE FROM students 
WHERE value > 6;

CREATE OR REPLACE TEMP VIEW updates(id, name, value, type) AS VALUES
  (2, "Omar", 15.2, "update"),
  (3, "", null, "delete"),
  (7, "Blue", 7.7, "insert"),
  (11, "Diya", 8.8, "update");
  
MERGE INTO students b
USING updates u
ON b.id=u.id
WHEN MATCHED AND u.type = "update"
  THEN UPDATE SET *
WHEN MATCHED AND u.type = "delete"
  THEN DELETE
WHEN NOT MATCHED AND u.type = "insert"
  THEN INSERT *;

In [0]:
DESCRIBE HISTORY students;

## テーブルの詳細を調べる

デフォルトでDatabricksはHiveメタストアを使用してスキーマ、テーブル、およびビューを登録します。

**`DESCRIBE EXTENDED`** を使用すると、テーブルに関する重要なメタデータを表示できます。

In [0]:
DESCRIBE EXTENDED students;

**`DESCRIBE DETAIL`** は、テーブルのメタデータを探索するための別のコマンドです。


In [0]:
DESCRIBE DETAIL students;

**`Location`** フィールドに注目してください。

これまで、私たちはテーブルを単なるスキーマ内の関係的なエンティティとして考えていましたが、Delta Lakeテーブルは実際にはクラウドオブジェクトストレージに格納されたファイルのコレクションでサポートされています。

## Delta Lakeファイルを探索する

Databricksのユーティリティ関数を使用して、Delta Lakeテーブルをサポートするファイルを見ることができます。

**注意**: Delta Lakeを操作するためにこれらのファイルのすべてを知る必要はありませんが、技術がどのように実装されているかを理解するのに役立ちます。

In [0]:
%python
tbl_location = spark.sql("DESCRIBE DETAIL students").first().location
data_files = dbutils.fs.ls(tbl_location)
display(data_files)


ディレクトリにはいくつかのParquetデータファイルと、 **`_delta_log`** という名前のディレクトリが含まれていることに注意してください。

Delta Lakeテーブルのレコードは、Parquetファイル内のデータとして保存されます。

Delta Lakeテーブルへのトランザクションは、 **`_delta_log`** に記録されます。

**`_delta_log`** 内部をのぞいて詳細を確認できます。

In [0]:
%python
log_files = dbutils.fs.ls(tbl_location + "/_delta_log")
display(log_files)

各トランザクションは、Delta Lakeトランザクションログに新しいJSONファイルが書き込まれる結果としています。ここでは、このテーブルに対して合計 10 のトランザクションがあることがわかります（Delta Lakeは0から始まるインデックスです）。  

★ Databricks Runtime 10.4 LTS 以降では、自動圧縮と最適化された書き込みが、MERGE、UPDATE、DELETE の各操作に対して常に有効
https://learn.microsoft.com/ja-jp/azure/databricks/delta/tune-file-size  

In [0]:
DESCRIBE HISTORY students;

## データファイルについての考察

明らかに非常に小さなテーブルに対して多くのデータファイルを見ました。

**`DESCRIBE DETAIL`** を使用すると、デルタテーブルに関する他の詳細を見ることができます。ファイルの数も含まれています。

In [0]:
DESCRIBE DETAIL students;

ここでは、該当テーブルにおける最新バージョンで有効なデータファイル数が確認できます。  

★ DeltaLakeは内部的にメタデータを保持しており _metadata.* 列としてアクセス可能

In [0]:
SELECT _metadata.file_name FROM students GROUP BY _metadata.file_name;

これはテーブルディレクトリにあるその他のParquetファイルの合計数よりも小さいのはなぜでしょう？

Delta Lakeは、変更されたデータを含むファイルを直ちに上書きまたは削除せず、トランザクションログを使用してファイルがテーブルの現在のバージョンで有効かどうかを示します。

ここでは、上記の **`MERGE`** ステートメントに対応するトランザクションログを見てみます。ここで、レコードが挿入、更新、削除されました。

In [0]:
%python
display(spark.sql(f"SELECT * FROM json.`{tbl_location}/_delta_log/00000000000000000008.json`"))

**`add`** 列には、テーブルに書き込まれたすべての新しいファイルのリストが含まれており、**`remove`** 列はもはやテーブルに含まれていてはいけないファイルを示しています。

Delta Lakeテーブルをクエリするとき、クエリエンジンはトランザクションログを使用して、現在のバージョンで有効なすべてのファイルを解決し、他のすべてのデータファイルを無視します。

In [0]:
INSERT INTO students VALUES (11, "oota", 1.1);
INSERT INTO students VALUES (12, "tomo", 2.2);
INSERT INTO students VALUES (13, "yuki", 3.3);

In [0]:
DESCRIBE HISTORY students;

In [0]:
SELECT _metadata.file_name FROM students GROUP BY _metadata.file_name;

## 小さなファイルの圧縮とインデックスの作成

小さなファイルはさまざまな理由で発生する可能性があります。私たちの場合、1つまたは数レコードだけが挿入された操作をいくつか実行しました。

ファイルは、**`OPTIMIZE`** コマンドを使用して最適なサイズに結合されます（テーブルのサイズに基づいてスケーリングされます）。

**`OPTIMIZE`** は、既存のデータファイルを置き換え、レコードを結合して結果を書き直します。

**`OPTIMIZE`** を実行する際、ユーザーは任意で**`ZORDER`** インデックスのための1つまたは複数のフィールドを指定できます。Z-orderの具体的な数学は重要ではありませんが、提供されたフィールドでフィルタリングを行う際にデータを同じ値を持つデータファイル内に配置することでデータの取得を高速化します。

In [0]:
OPTIMIZE students ZORDER BY id;

In [0]:
SELECT _metadata.file_name FROM students GROUP BY _metadata.file_name;

In [0]:
DESCRIBE HISTORY students;

予想通り、**`OPTIMIZE`** は私たちのテーブルの別のバージョンを作成し、バージョン8が最新のバージョンであることを意味します。

トランザクションログで削除されたとマークされた余分なデータファイルを覚えていますか？これらは、テーブルの以前のバージョンをクエリする機能を提供しています。

これらのタイムトラベルクエリは、整数のバージョンまたはタイムスタンプを指定して実行できます。

**注意**: ほとんどの場合、興味のある時点でデータを再作成するためにタイムスタンプを使用します。デモではバージョンを使用しますが、これは確定的です（将来の任意のタイミングでデモを実行する可能性があるため）。

In [0]:
SELECT * FROM students VERSION AS OF 3 ORDER BY id;

## バージョンをロールバックする

テーブルから一部のレコードを手動で削除するクエリを入力しようとして、次の状態でこのクエリを誤って実行したと仮定しましょう。

In [0]:
DELETE FROM students;

SELECT * FROM students;

テーブルのすべてのレコードを削除することはおそらく望ましくありません。幸い、このコミットを簡単にロールバックできます。

In [0]:
RESTORE TABLE students TO VERSION AS OF 13;

SELECT * FROM students ORDER BY id;

**`RESTORE`** <a href="https://docs.databricks.com/spark/latest/spark-sql/language-manual/delta-restore.html" target="_blank">コマンド</a> はトランザクションとして記録されます。テーブルのすべてのレコードを誤って削除した事実を完全に隠すことはできませんが、操作を元に戻し、テーブルを望ましい状態に戻すことはできます。

In [0]:
DESCRIBE HISTORY students;

In [0]:
%python
tbl_location = spark.sql("DESCRIBE DETAIL students").first().location
data_files = dbutils.fs.ls(tbl_location)
display(data_files)

## 古いファイルをクリーンアップする

DatabricksはDelta Lakeテーブル内の古いログファイル（デフォルトでは30日以上）を自動的にクリーンアップします。
チェックポイントが書き込まれるたびに、Databricksはこの保持期間よりも古いログエントリを自動的にクリーンアップします。

Delta Lakeのバージョニングとタイムトラベルは、最新バージョンのクエリとクエリのロールバックには適していますが、大規模な本番テーブルのすべてのバージョンのデータファイルを無期限に保持することは非常に高価です（また、PIIが存在する場合、コンプライアンスの問題につながる可能性があります）。

古いデータファイルを手動で削除する場合、これは **`VACUUM`** 操作で実行できます。

次のセルのコメントを解除し、 **`0 HOURS`** の保持期間で実行して、現在のバージョンのみを保持できます：

In [0]:
-- VACUUM students RETAIN 0 HOURS;

デフォルトでは、**`VACUUM`** は7日未満のファイルを削除しないようにします。これは、削除対象のファイルを参照している長時間実行される操作がないことを確認するためです。Deltaテーブルで**`VACUUM`** を実行すると、指定されたデータ保持期間よりも古いバージョンにタイムトラベルする能力が失われます。デモでは、**`0 HOURS`** の保持を指定するコードを実行する場面があるかもしれません。これは単にその機能をデモするためであり、通常は本番環境では行いません。

次のセルでは、次の操作を行います：
1. データファイルの早期削除を防ぐチェックを無効にする
2. **`VACUUM`** コマンドのログ記録を有効にする
3. ドライランバージョンの **`VACUUM`** を使用して削除されるすべてのレコードを表示します

In [0]:
ALTER TABLE students SET TBLPROPERTIES ('delta.deletedFileRetentionDuration' = '0 HOURS');

In [0]:
VACUUM students RETAIN 0 HOURS DRY RUN;

In [0]:
%python
tbl_location = spark.sql("DESCRIBE DETAIL students").first().location
data_files = dbutils.fs.ls(tbl_location)
display(data_files)

**`VACUUM`** を実行し、上記の10ファイルを削除することで、これらのファイルが必要とされるテーブルのバージョンへのアクセスが永久に削除されます。

In [0]:
VACUUM students RETAIN 0 HOURS;

In [0]:
%python
tbl_location = spark.sql("DESCRIBE DETAIL students").first().location
data_files = dbutils.fs.ls(tbl_location)
display(data_files)

# マネージドテーブル と アンマネージドテーブル

In [0]:
USE CATALOG hive_metastore;
CREATE SCHEMA IF NOT EXISTS ${your_schema};
USE SCHEMA ${your_schema};

## マネージドテーブル

In [0]:
CREATE OR REPLACE TABLE managed_table (width INT, length INT, height INT);
INSERT INTO managed_table VALUES (3, 2, 1);
SELECT * FROM managed_table;

In [0]:
DESCRIBE DETAIL managed_table;

In [0]:
DESCRIBE EXTENDED managed_table;

In [0]:
%python 
tbl_location = spark.sql("DESCRIBE DETAIL managed_table").first().location
print("location = " + tbl_location)

# マネージドディレクトリに対するリストコマンドの実行は Unity Catalog では未サポート
files = dbutils.fs.ls(tbl_location)
display(files)

ファイルは外部クラウドストレージに Delta Lake フォーマットで格納されます（通常はユーザーがストレージに直接アクセスすることは推奨されません）。
</br><img src="../images/deltalake.1.png" width="600"/>  

マネージドテーブルはテーブルの削除によりストレージ上のデータも削除されます（ユーザーがストレージを意識することはありません）。  

なお、データ削除のタイミングはテーブル削除から 30 日以内にシステム側で処理されます。  

参考：[マネージド テーブルの操作](https://learn.microsoft.com/ja-jp/azure/databricks/tables/managed)

In [0]:
DROP TABLE managed_table;

In [0]:
%python 

# マネージドディレクトリに対するリストコマンドの実行は Unity Catalog では未サポート
files = dbutils.fs.ls("dbfs:/user/hive/warehouse/handson_tico_demo01_schema.db/")
display(files)

## アンマネージドテーブル（外部テーブル）
次に、サンプルデータから　**外部**　（非管理）テーブルを作成します。

使用するデータはCSV形式です。指定したディレクトリ内の　**`LOCATION`**　を指定してDeltaテーブルを作成したいと思います。

In [0]:
-- ★これに必要な権限が不明。。。（いったんワークスペース管理者で通す → 汎用クラスタならOK）
CREATE OR REPLACE TEMPORARY VIEW temp_delays
  USING CSV OPTIONS (
    path = '${sample_dataset_path}/flights/departuredelays.csv',
    header = "true",
    mode = "FAILFAST" -- abort file parsing with a RuntimeException if any malformed lines are encountered
);

In [0]:

-- ★ UC の場合は外部ロケーションに対する CREATE EXTERNAL TABLE 権限が必要＆外部ロケーションとしてdbfsは未サポート
CREATE OR REPLACE TABLE external_table LOCATION 'dbfs:/user/hive/warehouse/handson_tico_demo01_schema.db/external_table/' AS
  SELECT * FROM temp_delays;

SELECT * FROM external_table LIMIT 10; 

In [0]:
DESCRIBE DETAIL external_table;

In [0]:
DESCRIBE TABLE EXTENDED external_table;

In [0]:
%python
# マネージドディレクトリに対するリストコマンドの実行は Unity Catalog では未サポート
files = dbutils.fs.ls('dbfs:/user/hive/warehouse/handson_tico_demo01_schema.db/external_table/')
display(files)

In [0]:
DROP TABLE external_table;

In [0]:
%python 
# マネージドディレクトリに対するリストコマンドの実行は Unity Catalog では未サポート
files = dbutils.fs.ls('dbfs:/user/hive/warehouse/handson_tico_demo01_schema.db/')
display(files)

アンマネージドテーブルはテーブルの削除を行ってもストレージ上のデータは保持されます（Databricks 上のメタデータ管理とストレージ上の実データ管理は切り離されています）。  
</br><img src="../images/deltalake.2.png" width="600"/>  

In [0]:
drop schema handson_tico_demo01_schema cascade;

# CTAS によるテーブル作成

In [0]:
USE CATALOG ${your_catalog};
USE SCHEMA ${your_schema};

**`CREATE TABLE AS SELECT`** ステートメントは、入力クエリから取得したデータを使用してDeltaテーブルを作成し、データを追加します。

In [0]:
CREATE OR REPLACE TABLE sales AS
SELECT * FROM parquet.`${sample_dataset_path}/ecommerce/raw/sales-historical`;

DESCRIBE EXTENDED sales;

CTASステートメントは、クエリの結果からスキーマ情報を自動的に推論し、手動でスキーマを宣言することは**サポートされていません**。

これは、CTASステートメントがスキーマが明示的に定義されたソースからの外部データ取り込みに適していることを意味します。例えば、Parquetファイルやテーブルなどです。

CTASステートメントはまた、追加のファイルオプションを指定することもサポートしていません。

CSVファイルからデータを取り込む際に、これが大きな制約となることが想像できます。

In [0]:
CREATE OR REPLACE TABLE sales_unparsed AS
SELECT * FROM csv.`${sample_dataset_path}/ecommerce/raw/sales-csv`;

SELECT * FROM sales_unparsed LIMIT 10;

このデータを正しくDelta Lakeテーブルに取り込むために、オプションを指定できるファイルへの参照を使用する必要があります。

前のレッスンでは、これを外部テーブルを登録することで行う方法を示しました。ここでは、この構文をわずかに進化させて、オプションを一時ビューに指定し、その後この一時ビューをCTASステートメントのソースとして使用してDeltaテーブルを正常に登録します。

In [0]:
CREATE OR REPLACE TEMP VIEW sales_tmp_vw
  (order_id LONG, email STRING, transactions_timestamp LONG, total_item_quantity INTEGER, purchase_revenue_in_usd DOUBLE, unique_items INTEGER, items STRING)
USING CSV
OPTIONS (
  path = "${sample_dataset_path}/ecommerce/raw/sales-csv",
  header = "true",
  delimiter = "|"
);

CREATE OR REPLACE TABLE sales_delta AS
  SELECT * FROM sales_tmp_vw;
  
SELECT * FROM sales_delta LIMIT 10;

## 既存のテーブルから列をフィルタリングおよび名前変更

列の名前を変更したり、対象のテーブルから列を省略したりするような簡単な変換は、テーブルの作成中に簡単に行うことができます。

次のステートメントは、 **`sales`** テーブルからの列のサブセットを含む新しいテーブルを作成します。

ここでは、意図的にユーザーを特定する可能性のある情報や個別の購入詳細を提供する情報を省略していると仮定します。また、下流のシステムがソースデータと異なる命名規則を持っていると仮定し、フィールド名を変更します。

In [0]:
CREATE OR REPLACE TABLE purchases AS
SELECT order_id AS id, transaction_timestamp, purchase_revenue_in_usd AS price
FROM sales;

SELECT * FROM purchases LIMIT 10;

同じ目標をビューを使用して達成することもできることに注意してください。以下に示すように。

In [0]:
CREATE OR REPLACE VIEW purchases_vw AS
SELECT order_id AS id, transaction_timestamp, purchase_revenue_in_usd AS price
FROM sales;

SELECT * FROM purchases_vw LIMIT 10;

## 生成された列を使用したスキーマの宣言

前述のように、CTASステートメントはスキーマの宣言をサポートしていません。上記で、タイムスタンプ列はUnixタイムスタンプのバリアントのようであり、アナリストが洞察を得るのに最も役立つ情報ではないようです。これは、生成された列が役立つ状況です。

生成された列は、Deltaテーブル内の他の列に基づいて自動的に生成される値を持つ特別なタイプの列です（DBR 8.3で導入）。

以下のコードは、新しいテーブルを作成しながら以下の操作を行います：
1. 列名と型を指定する
1. <a href="https://learn.microsoft.com/ja-jp/azure/databricks/delta/generated-columns" target="_blank">生成された列</a>を追加して日付を計算する
1. 生成された列の説明的な列コメントを提供する

In [0]:
CREATE OR REPLACE TABLE purchase_dates (
  id STRING, 
  transaction_timestamp STRING, 
  price STRING,
  date DATE GENERATED ALWAYS AS (
    cast(cast(transaction_timestamp/1e6 AS TIMESTAMP) AS DATE))
    COMMENT "generated based on `transactions_timestamp` column")

**`date`** は生成された列であるため、 **`date`** 列の値を提供せずに **`purchase_dates`** に書き込む場合、Delta Lakeは自動的に計算します。

★ WITH SCHEMA EVOLUTION によるスキーマの自動更新  
https://learn.microsoft.com/ja-jp/azure/databricks/delta/update-schema

**注意**: 以下のセルでは、Delta Lake **`MERGE`** ステートメントを使用する際に列を生成できるように設定を行っています。この構文については、コースの後半で詳しく説明します。

In [0]:
MERGE WITH SCHEMA EVOLUTION INTO purchase_dates a
USING purchases b
ON a.id = b.id
WHEN NOT MATCHED THEN
  INSERT *

以下から、データが挿入されるにつれてすべての日付が正しく計算されたことがわかります。ただし、ソースデータや挿入クエリはこのフィールドの値を指定していません。

Delta Lakeソースの場合、クエリは常にクエリのためのテーブルの最新のスナップショットを自動的に読み取ります。したがって、 **`REFRESH TABLE`** を実行する必要はありません。

In [0]:
SELECT * FROM purchase_dates LIMIT 10;

重要なことは、生成されたフィールドであるはずのフィールドがテーブルへの挿入に含まれている場合、提供された値が生成された列を定義するロジックで導出される値と完全に一致しない場合、この挿入は失敗することです。

以下のセルをコメント解除して実行することで、このエラーを確認できます。

In [0]:
-- INSERT INTO purchase_dates VALUES (1, 600000000, 42.0, "2020-06-18");

In [0]:
INSERT INTO purchase_dates VALUES (1, 1592237293397767, 1.0, cast(cast(1592237293397767/1e6 AS TIMESTAMP) AS DATE));

## テーブル制約の追加

上記のエラーメッセージは、 **`CHECK制約`** を指しています。生成された列は、チェック制約の特別な実装です。

Delta Lakeはスキーマの書き込み時に強制するため、Databricksはテーブルに追加されるデータの品質と整合性を確保するための標準SQL制約管理節をサポートできます。

Databricksは現在、2つのタイプの制約をサポートしています:
* <a href="https://docs.databricks.com/delta/delta-constraints.html#not-null-constraint" target="_blank">**`NOT NULL`** 制約</a>
* <a href="https://docs.databricks.com/delta/delta-constraints.html#check-constraint" target="_blank">**`CHECK`** 制約</a>

どちらの場合も、制約を定義する前に、制約に違反するデータが既にテーブルに存在しないことを確認する必要があります。一度制約がテーブルに追加されると、制約に違反するデータは書き込みエラーを引き起こします。

以下では、テーブルの **`date`** 列に **`CHECK`** 制約を追加します。 **`CHECK`** 制約は、データセットをフィルタリングするために使用するかもしれない標準の **`WHERE`** 句のように見えます。

In [0]:
ALTER TABLE purchase_dates ADD CONSTRAINT valid_date CHECK (date > '2020-01-01');

テーブル制約は、 **`TBLPROPERTIES`** フィールドに表示されます。

In [0]:
DESCRIBE EXTENDED purchase_dates

In [0]:
SELECT cast(cast(1572237293397767/1e6 AS TIMESTAMP) AS DATE);

In [0]:
-- INSERT INTO purchase_dates VALUES
-- (2, 1572237293397767, 2.0, cast(cast(1572237293397767/1e6 AS TIMESTAMP) AS DATE))

## 付加的なオプションとメタデータを使用してテーブルを拡張

これまで、Delta Lake テーブルを拡張するためのオプションについてほんの一部しか触れていません。

以下では、CTAS ステートメントを進化させ、さまざまな追加の構成とメタデータを含める方法を示します。

**`SELECT`** 句では、ファイルの取り込みに役立つ2つの組み込み Spark SQL コマンドを活用しています：
* **`current_timestamp()`** は、ロジックが実行されたタイムスタンプを記録します
* **`input_file_name()`** は、テーブル内の各レコードのソースデータファイルを記録します

また、ソースのタイムスタンプデータから派生した新しい日付列を作成するロジックも含まれています。

**`CREATE TABLE`** 句にはいくつかのオプションが含まれています：
* テーブルの内容を簡単に見つけるために **`COMMENT`** が追加されています
* **`LOCATION`** が指定されており、管理されていない（管理されていないではなく、外部の）テーブルになります
* テーブルは日付列によって **`PARTITIONED BY`** されており、各日付のデータが対象のストレージ場所内の独自のディレクトリに存在することを意味します

**注意**: パーティショニングは、主に構文と影響を示すためにここに示されています。ほとんどの Delta Lake テーブル（特に小～中規模のデータ）はパーティショニングの恩恵を受けません。パーティショニングはデータファイルを物理的に分離するため、小さなファイルの問題を引き起こし、ファイルの圧縮と効率的なデータのスキップを妨げる可能性があります。Hive または HDFS で観察されるメリットは Delta Lake には適用されず、テーブルをパーティション分割する前に経験豊富な Delta Lake アーキテクトと相談する必要があります。

**ほとんどのユースケースでは、Delta Lake で作業する際には非パーティションテーブルをデフォルトで選択すべきです。**

In [0]:
CREATE OR REPLACE TABLE users_pii
COMMENT "Contains PII"
AS
  SELECT *, 
    cast(cast(user_first_touch_timestamp/1e6 AS TIMESTAMP) AS DATE) first_touch_date, 
    current_timestamp() updated,
    input_file_name() source_file
  FROM parquet.`${sample_dataset_path}/ecommerce/raw/users-historical/`;
  
SELECT * FROM users_pii LIMIT 10;

In [0]:
SELECT COUNT(*) AS count, source_file FROM users_pii GROUP BY source_file

テーブルに追加されたメタデータフィールドは、レコードがいつ挿入されたか、どこから来たかを理解するのに役立つ情報を提供します。これは、ソースデータの問題をトラブルシューティングする必要がある場合に特に役立ちます。

与えられたテーブルのすべてのコメントとプロパティは、 **`DESCRIBE TABLE EXTENDED`** を使用して確認できます。

**注意**: Delta Lakeは、テーブル作成時に自動的にいくつかのテーブルプロパティを追加します。

In [0]:
DESCRIBE EXTENDED users_pii

## Delta Lakeテーブルのクローン
Delta LakeにはDelta Lakeテーブルを効率的にコピーするための2つのオプションがあります。

**`DEEP CLONE`** は、データとメタデータをソーステーブルからターゲットに完全にコピーします。
このコピーは段階的に行われるため、このコマンドを再実行すると、ソースからターゲットの場所への変更が同期されます。

In [0]:
CREATE OR REPLACE TABLE purchases_clone DEEP CLONE purchases;

すべてのデータファイルをコピーする必要があるため、大規模なデータセットにはかなりの時間がかかることがあります。

現在のテーブルを変更せずに変更を適用してテストするためにテーブルのコピーを迅速に作成したい場合、 **`SHALLOW CLONE`** は良いオプションです。
シャロークローンはDeltaトランザクションログだけをコピーするため、データは移動しません。

In [0]:
CREATE OR REPLACE TABLE purchases_shallow_clone SHALLOW CLONE purchases;

どちらの場合でも、テーブルのクローンに適用されるデータの変更は、ソースから別個に追跡および保存されます。 クローンは、開発中のSQLコードのテスト用にテーブルをセットアップする素晴らしい方法です。

★ 時間あればもう少しクローンを説明する  
https://adb-1450470117424213.13.azuredatabricks.net/editor/notebooks/1625313471164596?o=1450470117424213#command/6519521898549652

# Delta Lakeへのデータの読み込み
Delta Lakeテーブルは、クラウドオブジェクトストレージでバックアップされたデータファイルを持つテーブルにACID準拠の更新を提供します。

このノートブックでは、Delta Lakeで更新を処理するためのSQL構文を探求します。多くの操作は標準のSQLですが、SparkとDelta Lakeの実行に合わせてわずかな変更があります。

## 学習目標
このレッスンの最後までに、次のことができるようになるはずです：
- **`INSERT OVERWRITE`** を使用してデータテーブルを上書きする
- **`INSERT INTO`** を使用してテーブルに追加する
- **`MERGE INTO`** を使用してテーブルに追加、更新、削除を行う
- **`COPY INTO`** を使用してテーブルにデータを増分的に読み込む

## 完全な上書き

テーブル内のすべてのデータを原子的に置き換えるために上書きを使用できます。テーブルを削除して再作成する代わりにテーブルを上書きすることには、複数の利点があります：
- テーブルを上書きする方がはるかに高速です。ディレクトリを再帰的にリストアップしたり、ファイルを削除したりする必要がないためです。
- テーブルの古いバージョンはまだ存在し、タイムトラベルを使用して簡単に古いデータを取得できます。
- これは原子操作です。テーブルを削除している間でも、同時クエリはテーブルを読み取ることができます。
- ACIDトランザクションの保証により、テーブルの上書きに失敗した場合、テーブルは以前の状態になります。

Spark SQLは完全な上書きを実行するための2つの簡単な方法を提供しています。

前のレッスンでCTASステートメントについて学んだ学生の中には、実際にはCRASステートメントを使用したことに気付いたかもしれません（セルが複数回実行された場合の潜在的なエラーを回避するためです）。

**`CREATE OR REPLACE TABLE`** （CRAS）ステートメントは、実行ごとにテーブルの内容を完全に置き換えます。

In [0]:
CREATE OR REPLACE TABLE events AS
SELECT * FROM parquet.`${sample_dataset_path}/ecommerce/raw/events-historical`;

SELECT count(*) FROM events;

**`INSERT OVERWRITE`** は、上記とほぼ同じ結果を提供します：ターゲットテーブル内のデータはクエリからのデータで置き換えられます。

**`INSERT OVERWRITE`**：

- 既存のテーブルを上書きすることしかできず、CRASステートメントのように新しいテーブルを作成することはできません
- 現在のテーブルスキーマに一致する新しいレコードでのみ上書きでき、したがって、下流の消費者を中断せずに既存のテーブルを上書きする「安全な」テクニックとなります
- 個々のパーティションを上書きできます

In [0]:
INSERT OVERWRITE events SELECT * FROM parquet.`${sample_dataset_path}/ecommerce/raw/events-historical`;

SELECT count(*) FROM events;

テーブルの履歴を確認すると、以前のバージョンのこのテーブルが置き換えられたことがわかります。  
CRASステートメントとは異なるメトリクスが表示されることに注意してください。また、テーブルの履歴も操作を異なる方法で記録します。  

In [0]:
DESCRIBE HISTORY events

## 行の追加

**`INSERT INTO`** を使用して、既存のDeltaテーブルに新しい行を原子的に追加できます。これにより、毎回上書きするよりも効率的な既存のテーブルへの増分更新が可能になります。

**`INSERT INTO`** を使用して、**`sales`** テーブルに新しい販売レコードを追加します。

In [0]:
CREATE OR REPLACE TABLE sales AS
  SELECT * FROM parquet.`${sample_dataset_path}/ecommerce/raw/sales-historical`;

SELECT count(*), _metadata.file_path FROM sales GROUP BY _metadata.file_path;

In [0]:
INSERT INTO sales SELECT * FROM parquet.`${sample_dataset_path}/ecommerce/raw/sales-30m`;

SELECT count(*), _metadata.file_path FROM sales GROUP BY _metadata.file_path;

**`INSERT INTO`** には、同じレコードを複数回挿入するのを防ぐための組み込みの保証がないことに注意してください。上記のセルを再実行すると、同じレコードがターゲットテーブルに書き込まれ、重複したレコードが生成されます。

In [0]:
INSERT INTO sales SELECT * FROM parquet.`${sample_dataset_path}/ecommerce/raw/sales-30m`;

SELECT count(*), _metadata.file_path FROM sales GROUP BY _metadata.file_path;

## 増分的な読み込み

**`COPY INTO`** はSQLエンジニアに対して外部システムからデータを増分的に読み込むための冪等性のあるオプションを提供します。

この操作にはいくつかの期待があることに注意してください：
- データスキーマは一貫している必要があります
- 重複するレコードは排除または下流で処理する必要があります

この操作は、予測可能に成長するデータに対して完全なテーブルスキャンよりもはるかに安価かもしれません。

ここでは静的なディレクトリでの単純な実行を示しますが、実際の価値は時間をかけて複数回実行し、ソースから新しいファイルを自動的に収集することにあります。

In [0]:
CREATE OR REPLACE TABLE sales AS
  SELECT * FROM parquet.`${sample_dataset_path}/ecommerce/raw/sales-historical`;

SELECT count(*), _metadata.file_path FROM sales GROUP BY _metadata.file_path;

In [0]:
COPY INTO sales FROM "${sample_dataset_path}/ecommerce/raw/sales-30m" FILEFORMAT = PARQUET;

SELECT count(*), _metadata.file_path FROM sales GROUP BY _metadata.file_path;

Databricks の COPY INTO コマンドの冪等性は、以下の方法で保証されています。

**ファイルの追跡**: COPY INTO コマンドは、入力ファイルの詳細を Delta テーブルのログディレクトリ内に保存します。この情報は RocksDB というキー・バリュー・ストアに格納されます。  
**重複除去ロジック**: 次回 COPY INTO コマンドが同じテーブルに対して実行されると、まず RocksDB からデータが読み込まれ、入力ファイルと比較されます。この比較により、既にロードされたファイルはスキップされます。  
**強制オプション**: COPY_OPTIONS の force パラメータを true に設定すると、冪等性が無効化され、以前にロードされたファイルも再度ロードされます。  

このようにして、COPY INTO コマンドは同じデータを複数回ロードしても重複しないように設計されています。

https://community.databricks.com/t5/machine-learning/how-is-idempotency-ensured-for-copy-into-command/td-p/19795

In [0]:
COPY INTO sales FROM "${sample_dataset_path}/ecommerce/raw/sales-30m" FILEFORMAT = PARQUET;

SELECT count(*), _metadata.file_path FROM sales GROUP BY _metadata.file_path;

## マージ更新

**`MERGE`** SQL操作を使用して、ソーステーブル、ビュー、またはDataFrameからターゲットDeltaテーブルにデータをアップサートできます。 Delta Lakeは **`MERGE`** での挿入、更新、削除をサポートし、高度なユースケースを簡素化するためのSQL標準を超えた拡張構文をサポートしています。

<strong><code>
MERGE INTO target a<br/>
USING source b<br/>
ON {merge_condition}<br/>
WHEN MATCHED THEN {matched_action}<br/>
WHEN NOT MATCHED THEN {not_matched_action}<br/>
</code></strong>

私たちは**`MERGE`**操作を使用して、更新されたメールアドレスと新しいユーザーを持つ過去のユーザーデータを更新します。


In [0]:
CREATE OR REPLACE TEMP VIEW users_update 
AS 
  SELECT *,
    cast(cast(user_first_touch_timestamp/1e6 AS TIMESTAMP) AS DATE) first_touch_date, 
    current_timestamp() AS updated,
    input_file_name() source_file
  FROM parquet.`${sample_dataset_path}/ecommerce/raw/users-30m`;

SELECT count(*) FROM users_update;

**`MERGE`** の主な利点：
* 更新、挿入、削除が1つのトランザクションとして完了します
* マッチングフィールドに加えて複数の条件を追加できます
* カスタムロジックを実装するための幅広いオプションを提供します

以下では、現在の行の**`NULL`**のメールアドレスと新しい行がメールアドレスを持っている場合にのみレコードを更新します。

新しいバッチのすべてのマッチしないレコードは挿入されます。

In [0]:
MERGE INTO users_pii a
USING users_update b
ON a.user_id = b.user_id
WHEN MATCHED AND a.email IS NULL AND b.email IS NOT NULL THEN
  UPDATE SET email = b.email, updated = b.updated
WHEN NOT MATCHED THEN
  INSERT *

In [0]:
SELECT COUNT(*) AS count, source_file FROM users_pii GROUP BY source_file

この関数の動作を **`MATCHED`** および **`NOT MATCHED`** の両条件に対して明示的に指定しています。ここで示されている例は適用できるロジックの一例であり、 **`MERGE`** のすべての動作を示すものではありません。